In [1]:
## let's start
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
# read data
with open ('input_abcd.txt', 'r', encoding='utf-8') as f:
    text=f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  244720


In [4]:
print(text[:10])

abcd abcd 


In [5]:
# unique characters
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)

 abcd
5


In [7]:
# create character to integer mapping
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # input string and output integers
decode = lambda l: ''.join([itos[i] for i in l]) # input integers and outpu string

#print(encode("hii there"))
#print(decode(encode("hii there")))

In [8]:
#question for future 1: large vocab/small sequence how is the data read to split into tokens

In [9]:
#encode entire text and store into torch tensor
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # first 100 characters - encoded

torch.Size([244720]) torch.int64
tensor([1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
        2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
        1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
        4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2,
        3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1,
        2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0,
        1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4,
        0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 0, 1, 2, 3,
       

In [10]:
# split data into training and validation set
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [11]:
block_size=8
train_data[:block_size+1]

tensor([1, 2, 3, 4, 0, 1, 2, 3, 4])

In [12]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 2
when input is tensor([1, 2]) the target: 3
when input is tensor([1, 2, 3]) the target: 4
when input is tensor([1, 2, 3, 4]) the target: 0
when input is tensor([1, 2, 3, 4, 0]) the target: 1
when input is tensor([1, 2, 3, 4, 0, 1]) the target: 2
when input is tensor([1, 2, 3, 4, 0, 1, 2]) the target: 3
when input is tensor([1, 2, 3, 4, 0, 1, 2, 3]) the target: 4


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F


#hyperparameters
batch_size=64 # independent sequences in parallel
block_size=128 # context length
max_iters=5000
eval_interval=500
learning_rate=3e-4
device ='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
n_embd=192
n_head=6
n_layer=6
dropout=0.1

#--------------------
torch.manual_seed(1337)


def get_batch(split):
    #generate a small batch of inputs x and targets y
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y=x.to(device),y.to(device)
    return x,y

xb, yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

# for i in range(batch_size):  #batch dimension - rows
#     for t in range(block_size): #time dimension - column
#         context=xb[i,:t+1]
#         target=yb[i,t]
#         print(f"when input is {context} output is: {target}")


inputs:
torch.Size([64, 128])
tensor([[1, 2, 3,  ..., 1, 2, 3],
        [3, 4, 0,  ..., 3, 4, 0],
        [3, 4, 0,  ..., 3, 4, 0],
        ...,
        [2, 3, 4,  ..., 2, 3, 4],
        [3, 4, 0,  ..., 3, 4, 0],
        [2, 3, 4,  ..., 2, 3, 4]], device='cuda:0')
targets:
torch.Size([64, 128])
tensor([[2, 3, 4,  ..., 2, 3, 4],
        [4, 0, 1,  ..., 4, 0, 1],
        [4, 0, 1,  ..., 4, 0, 1],
        ...,
        [3, 4, 0,  ..., 3, 4, 0],
        [4, 0, 1,  ..., 4, 0, 1],
        [3, 4, 0,  ..., 3, 4, 0]], device='cuda:0')


In [14]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y=get_batch(split)
            X,Y=X.to(device),Y.to(device)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out


In [15]:


class Head(nn.Module):
    """One head of self attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embd,head_size,bias=False)
        self.query=nn.Linear(n_embd,head_size,bias=False)
        self.value=nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        B,T,C = x.shape
        k=self.key(x) # (B,T,C head_size)
        q=self.query(x) # (B,T,C head_size)
        # compute attention scores - affinities
        wei = q @ k.transpose(-2,-1) *C**-0.5 # (B,T,16) * (B,16,T) --> (B,T,T)
        wei=wei.masked_fill(self.tril[:T,:T]==0,float('-inf')) #(B,T,T)
        wei=F.softmax(wei,dim=-1) #(B,T,T)
        wei=self.dropout(wei)
        # perform weighted aggregation of the values
        v=self.value(x) # (B,T,C)
        out=wei @ v #(B,T,T) @ (B,T,C) -> (B,T,C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self attention in parallel"""

    def __init__(self, num_heads,head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd,n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x):
        out=torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [16]:
class FeedForward(nn.Module):
    """a simple linear layer followed by non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd), #projection layer, added in multi head attention
            nn.Dropout(dropout)
        )
    
    def forward(self,x):
        return self.net(x)
    
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd,n_head):
        #n_embd - number of embeddings, n_head = number of heads
        super().__init__()
        head_size = n_embd//n_head
        self.sa_heads=MultiHeadAttention(n_head,head_size) # heads of dimensional self-attention
        self.ffwd=FeedForward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)

    def forward(self,x):
        x=x + self.sa_heads(self.ln1(x)) # apply 4 heads of 8 dimensional self attention (B,T,C)
        x=x + self.ffwd(self.ln2(x)) # (B, T, C)  
        return x

In [17]:
# backup

# class BigramLanguageModel(nn.Module):
    
#     def __init__(self):
#         super().__init__()
#         #each token directly reads off the logits for the next token from a lookup table
#         self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
#         self.position_embedding_table=nn.Embedding(block_size,n_embd)
#         #self.sa_head=Head(n_embd)
#         self.sa_heads=MultiHeadAttention(4,n_embd//4) # 4 heads of 8 dimensional self-attention
#         self.ffwd=FeedForward(n_embd)
#         self.lm_head=nn.Linear(n_embd,vocab_size)
    
#     def forward(self,idx,targets=None):
#         B,T = idx.shape
#         #idx and targets are both B,T tensor of integers
#         tok_emb= self.token_embedding_table(idx) #(B,T,C)
#         pos_emb=self.position_embedding_table(torch.arange(T,device=device)) #(T,C)
#         x = tok_emb + pos_emb #(B,T,C)
#         #x=self.sa_head(x) # apply one head of self attention (B,T,C)
#         x=self.sa_heads(x) # apply 4 heads of 8 dimensional self attention (B,T,C)
#         x=self.ffwd(x) # (B, T, C)
#         logits=self.lm_head(x) #(B,T,vocab_size)   

#         if targets is None:
#             loss = None
#         else: 
#             B,T,C=logits.shape
#             logits=logits.view(B*T,C)
#             targets=targets.view(B*T)
#             loss=F.cross_entropy(logits,targets)

#         return logits,loss

In [18]:





class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table=nn.Embedding(block_size,n_embd)
        # self.blocks = nn.Sequential(
        #     Block(n_embd,n_head=4),
        #     Block(n_embd,n_head=4),
        #     Block(n_embd,n_head=4),
        #     nn.LayerNorm(n_embd)
        # )
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) # final layer norm
        self.lm_head=nn.Linear(n_embd,vocab_size)
    
    def forward(self,idx,targets=None):
        B,T = idx.shape
        #idx and targets are both B,T tensor of integers
        tok_emb= self.token_embedding_table(idx) #(B,T,C)
        pos_emb=self.position_embedding_table(torch.arange(T,device=device)) #(T,C)
        x = tok_emb + pos_emb #(B,T,C)
        #x=self.sa_head(x) # apply one head of self attention (B,T,C)
        x=self.blocks(x) #(B,T,C)
        x=self.ln_f(x) #(B,T,C)
        logits=self.lm_head(x) #(B,T,vocab_size)   

        if targets is None:
            loss = None
        else: 
            B,T,C=logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)

        return logits,loss
    
    def generate(self,idx,max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to last block_size tokens
            idx_cond=idx[:,-block_size:]
            # get the predictions
            logits,loss =self(idx_cond)
            # focus only on the last time step
            logits=logits[:,-1,:] #becomes B,C
            # apply softmax to get the probabilities
            probs=F.softmax(logits,dim=-1) # B,C
            # sample from the distribution
            idx_next=torch.multinomial(probs,num_samples=1) # B,1
            # append sample index to the running sequence
            idx=torch.cat((idx, idx_next),dim=1) #(B,T+1)
        return idx


model=BigramLanguageModel()
model=model.to(device)
#out=m(xb,yb)
logits,loss=model(xb,yb)
print(logits.shape)
print(loss)

idx=torch.zeros((1,1),dtype=torch.long)
#print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([8192, 5])
tensor(1.7590, device='cuda:0', grad_fn=<NllLossBackward0>)


In [19]:
# create a pytorch optimizer
optimizer=torch.optim.AdamW(model.parameters(),lr=1e-3)

In [20]:
#batch_size=32
for iter in range(max_iters):

    # evey once in a while evaluate the loss on train and val sets
    if iter % eval_interval==0:
        losses=estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss{losses['val']:.4f}")

    # sample a batch of data
    (xb,yb)=get_batch('train')

    # evaluate the loss
    logits,loss=model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step 0: train loss 1.7644, val loss1.7645
step 500: train loss 0.0001, val loss0.0001
step 1000: train loss 0.0000, val loss0.0000
step 1500: train loss 0.0000, val loss0.0000
step 2000: train loss 0.0000, val loss0.0000
step 2500: train loss 0.0000, val loss0.0000
step 3000: train loss 0.0000, val loss0.0000
step 3500: train loss 0.0000, val loss0.0000
step 4000: train loss 0.0000, val loss0.0000
step 4500: train loss 0.0000, val loss0.0000
1.4253890867621521e-06


In [21]:
context=torch.zeros((1,1),dtype=torch.long,device=device)
print(decode(model.generate(context,max_new_tokens=100)[0].tolist()))

 abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd 


## Self Attention test

In [23]:
torch.manual_seed(1337)
B,T,C=4,8,2 #batch, time, channels
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [24]:
# version 1
xbow=torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1]
        xbow[b,t]=torch.mean(xprev,0)
        
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [25]:
# version 2
wei = torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1, keepdim=True)
wei
xbow2=wei @ x # wei-> B * T * T  x -> B * T * C, output B * T * C
xbow2[0]
torch.allclose(xbow,xbow2)

False

In [26]:
# version 3
tril = torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)

xbow3=wei @ x
torch.allclose(xbow2,xbow3)


True

In [ ]:
# version 4: self attention
torch.manual_seed(1337)
B,T,C=4,8,32 #batch, time, channels
x=torch.randn(B,T,C)

# single head
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)

k=key(x) # (B,T,16 head_size)
q=query(x) # (B,T,16 head_size)
wei = q @ k.transpose(-2,-1) # (B,T,16) * (B,16,T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
#wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0,float('-inf'))
wei=F.softmax(wei,dim=-1)

v=value(x)
out=wei @ v

wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [33]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a=a/torch.sum(a,1, keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c=a@b
c

tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])

## Old test

In [ ]:
block_size=8 # context length
batch_size=4 # independent sequences in parallel
ix=torch.randint(len(data)-block_size,(batch_size,))
ix

In [1]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch version: 2.5.1+cu121
CUDA available: True


In [2]:
print(torch.cuda.get_device_name(0))
print(torch.rand(2,2).cuda())

NVIDIA GeForce RTX 4050 Laptop GPU
tensor([[0.6789, 0.4387],
        [0.5568, 0.1342]], device='cuda:0')
